# Soft Power Database Audit

Comprehensive audit of all database tables and metrics, with focus on:
- Documents and their distribution
- Event hierarchies (clusters, canonical events, daily mentions, summaries)
- Country analysis (influencers and recipients from config.yaml)
- Embeddings and vector stores
- Date coverage and temporal analysis

In [1]:
import sys
import os
from pathlib import Path

# ============================================================
# DATABASE CONNECTION CONFIGURATION
# ============================================================
# IMPORTANT: Set environment variables BEFORE any other imports
# to ensure the database module uses the correct host.
#
# When running Jupyter OUTSIDE Docker: use 'localhost'
# When running Jupyter INSIDE Docker: use 'softpower_db'
# ============================================================

# Set DB_HOST before any imports - this must come first!
os.environ['DB_HOST'] = 'localhost'  # Change to 'softpower_db' if running inside Docker
os.environ.pop('DATABASE_URL', None)  # Clear any cached DATABASE_URL

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import yaml
from datetime import datetime, date
from sqlalchemy import text, func
import warnings
warnings.filterwarnings('ignore')

# Force reload of database module to pick up new environment
import importlib
if 'shared.database.database' in sys.modules:
    # Clear existing db manager to force reconnection
    import shared.database.database as db_module
    db_module._db_manager = None
    importlib.reload(db_module)

# Now import the database functions
from shared.database.database import get_engine, get_session

# Load config
with open(project_root / 'shared/config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

INFLUENCERS = config.get('influencers', [])
RECIPIENTS = config.get('recipients', [])
CATEGORIES = config.get('categories', [])

print("Configuration Loaded")
print(f"  Influencing Countries: {INFLUENCERS}")
print(f"  Recipient Countries: {len(RECIPIENTS)} countries")
print(f"  Categories: {CATEGORIES}")
print(f"  DB Host: {os.environ.get('DB_HOST', 'NOT SET')}")

# Test connection
try:
    from shared.database.database import health_check
    if health_check():
        print("  Database Connection: OK")
    else:
        print("  Database Connection: FAILED")
except Exception as e:
    print(f"  Database Connection: ERROR - {e}")

In [2]:
# Helper function to run queries
def run_query(query, params=None):
    """Run a SQL query and return results as DataFrame."""
    engine = get_engine()
    with engine.connect() as conn:
        result = pd.read_sql(text(query), conn, params=params)
    return result

def run_scalar(query, params=None):
    """Run a SQL query and return scalar result."""
    engine = get_engine()
    with engine.connect() as conn:
        result = conn.execute(text(query), params or {}).scalar()
    return result

---
## 1. Table Overview

Summary of all tables and their row counts.

In [3]:
# Get all table counts
tables_query = """
SELECT 
    schemaname,
    relname as table_name,
    n_live_tup as row_count
FROM pg_stat_user_tables
ORDER BY n_live_tup DESC
"""

tables_df = run_query(tables_query)
print("=" * 60)
print("TABLE OVERVIEW")
print("=" * 60)
print(f"\nTotal tables: {len(tables_df)}")
print(f"Total rows across all tables: {tables_df['row_count'].sum():,}")
print("\n")
tables_df

ERROR:shared.database.database:Database connection attempt 1 failed: (psycopg2.OperationalError) could not translate host name "softpower_db" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)
ERROR:shared.database.database:Database connection attempt 2 failed: (psycopg2.OperationalError) could not translate host name "softpower_db" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)
ERROR:shared.database.database:Database connection attempt 3 failed: (psycopg2.OperationalError) could not translate host name "softpower_db" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)


ConnectionError: Failed to connect to database after 3 attempts: (psycopg2.OperationalError) could not translate host name "softpower_db" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)

---
## 2. Documents Analysis

In [5]:
print("=" * 60)
print("DOCUMENTS TABLE ANALYSIS")
print("=" * 60)

# Basic counts
doc_count = run_scalar("SELECT COUNT(*) FROM documents")
print(f"\nTotal Documents: {doc_count:,}")

# Date range
date_range = run_query("""
    SELECT 
        MIN(date) as earliest_date,
        MAX(date) as latest_date,
        COUNT(DISTINCT date) as unique_dates
    FROM documents
    WHERE date IS NOT NULL
""")
print(f"\nDate Range:")
print(f"  Earliest: {date_range['earliest_date'].iloc[0]}")
print(f"  Latest: {date_range['latest_date'].iloc[0]}")
print(f"  Unique Dates: {date_range['unique_dates'].iloc[0]:,}")

# Documents by month
docs_by_month = run_query("""
    SELECT 
        DATE_TRUNC('month', date)::date as month,
        COUNT(*) as doc_count
    FROM documents
    WHERE date IS NOT NULL
    GROUP BY DATE_TRUNC('month', date)
    ORDER BY month
""")
print(f"\nDocuments by Month:")
docs_by_month

ERROR:shared.database.database:Database connection attempt 1 failed: (psycopg2.OperationalError) could not translate host name "softpower_db" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)


DOCUMENTS TABLE ANALYSIS


ERROR:shared.database.database:Database connection attempt 2 failed: (psycopg2.OperationalError) could not translate host name "softpower_db" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)
ERROR:shared.database.database:Database connection attempt 3 failed: (psycopg2.OperationalError) could not translate host name "softpower_db" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)


ConnectionError: Failed to connect to database after 3 attempts: (psycopg2.OperationalError) could not translate host name "softpower_db" to address: Name or service not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
# Documents by initiating country (from config influencers)
print("\n" + "=" * 60)
print("DOCUMENTS BY INITIATING COUNTRY (Influencers)")
print("=" * 60)

init_country_query = """
    SELECT 
        initiating_country,
        COUNT(*) as doc_count,
        MIN(date) as earliest_date,
        MAX(date) as latest_date
    FROM documents
    WHERE initiating_country IS NOT NULL
    GROUP BY initiating_country
    ORDER BY doc_count DESC
"""
init_countries = run_query(init_country_query)

# Highlight config influencers
init_countries['in_config'] = init_countries['initiating_country'].isin(INFLUENCERS)
print("\nAll Initiating Countries:")
display(init_countries)

# Summary for config influencers
config_influencers = init_countries[init_countries['in_config']]
print(f"\nConfig Influencers Coverage:")
print(f"  Total docs from config influencers: {config_influencers['doc_count'].sum():,}")
print(f"  Percentage of total: {config_influencers['doc_count'].sum() / doc_count * 100:.1f}%")

In [ ]:
# Documents by recipient country (from config recipients)
print("\n" + "=" * 60)
print("DOCUMENTS BY RECIPIENT COUNTRY (Recipients)")
print("=" * 60)

rec_country_query = """
    SELECT 
        recipient_country,
        COUNT(*) as doc_count,
        MIN(date) as earliest_date,
        MAX(date) as latest_date
    FROM documents
    WHERE recipient_country IS NOT NULL
    GROUP BY recipient_country
    ORDER BY doc_count DESC
"""
rec_countries = run_query(rec_country_query)

# Highlight config recipients
rec_countries['in_config'] = rec_countries['recipient_country'].isin(RECIPIENTS)
print("\nTop 30 Recipient Countries:")
display(rec_countries.head(30))

# Summary for config recipients
config_recipients = rec_countries[rec_countries['in_config']]
print(f"\nConfig Recipients Coverage:")
print(f"  Total docs to config recipients: {config_recipients['doc_count'].sum():,}")
print(f"  Percentage of total: {config_recipients['doc_count'].sum() / doc_count * 100:.1f}%")

In [ ]:
# Documents by category
print("\n" + "=" * 60)
print("DOCUMENTS BY CATEGORY")
print("=" * 60)

category_query = """
    SELECT 
        category,
        COUNT(*) as doc_count
    FROM documents
    WHERE category IS NOT NULL
    GROUP BY category
    ORDER BY doc_count DESC
"""
categories = run_query(category_query)
categories['in_config'] = categories['category'].isin(CATEGORIES)
print("\nCategories:")
categories

In [ ]:
# Salience analysis
print("\n" + "=" * 60)
print("SALIENCE DISTRIBUTION")
print("=" * 60)

salience_query = """
    SELECT 
        salience_bool,
        COUNT(*) as doc_count
    FROM documents
    GROUP BY salience_bool
    ORDER BY doc_count DESC
"""
salience_dist = run_query(salience_query)
print("\nSalience Distribution:")
salience_dist

---
## 3. Normalized Relationship Tables

In [ ]:
print("=" * 60)
print("NORMALIZED RELATIONSHIP TABLES")
print("=" * 60)

# Categories table
cat_count = run_scalar("SELECT COUNT(*) FROM categories")
cat_unique = run_scalar("SELECT COUNT(DISTINCT category) FROM categories")
print(f"\nCategories Table:")
print(f"  Total rows: {cat_count:,}")
print(f"  Unique categories: {cat_unique}")

# Subcategories table
subcat_count = run_scalar("SELECT COUNT(*) FROM subcategories")
subcat_unique = run_scalar("SELECT COUNT(DISTINCT subcategory) FROM subcategories")
print(f"\nSubcategories Table:")
print(f"  Total rows: {subcat_count:,}")
print(f"  Unique subcategories: {subcat_unique}")

# Initiating countries table
init_count = run_scalar("SELECT COUNT(*) FROM initiating_countries")
init_unique = run_scalar("SELECT COUNT(DISTINCT initiating_country) FROM initiating_countries")
print(f"\nInitiating Countries Table:")
print(f"  Total rows: {init_count:,}")
print(f"  Unique countries: {init_unique}")

# Recipient countries table
rec_count = run_scalar("SELECT COUNT(*) FROM recipient_countries")
rec_unique = run_scalar("SELECT COUNT(DISTINCT recipient_country) FROM recipient_countries")
print(f"\nRecipient Countries Table:")
print(f"  Total rows: {rec_count:,}")
print(f"  Unique countries: {rec_unique}")

# Raw events table
raw_events_count = run_scalar("SELECT COUNT(*) FROM raw_events")
raw_events_unique = run_scalar("SELECT COUNT(DISTINCT event_name) FROM raw_events")
print(f"\nRaw Events Table:")
print(f"  Total rows: {raw_events_count:,}")
print(f"  Unique event names: {raw_events_unique:,}")

---
## 4. Event Hierarchy Analysis

The event processing pipeline creates multiple levels of event aggregation:
1. **Event Clusters** - Initial DBSCAN clustering by country/date
2. **Canonical Events** - LLM-validated unique events
3. **Daily Event Mentions** - Daily mentions of canonical events
4. **Event Summaries** - Period-based summaries (daily/weekly/monthly)

In [ ]:
print("=" * 60)
print("EVENT CLUSTERS ANALYSIS")
print("=" * 60)

# Check if table exists and has data
try:
    cluster_count = run_scalar("SELECT COUNT(*) FROM event_clusters")
    print(f"\nTotal Event Clusters: {cluster_count:,}")
    
    if cluster_count > 0:
        # Clusters by country
        clusters_by_country = run_query("""
            SELECT 
                initiating_country,
                COUNT(*) as cluster_count,
                SUM(cluster_size) as total_events,
                MIN(cluster_date) as earliest_date,
                MAX(cluster_date) as latest_date,
                SUM(CASE WHEN processed THEN 1 ELSE 0 END) as processed_count,
                SUM(CASE WHEN llm_deconflicted THEN 1 ELSE 0 END) as llm_validated_count
            FROM event_clusters
            GROUP BY initiating_country
            ORDER BY cluster_count DESC
        """)
        clusters_by_country['in_config'] = clusters_by_country['initiating_country'].isin(INFLUENCERS)
        print("\nClusters by Country:")
        display(clusters_by_country)
        
        # Clusters by month
        clusters_by_month = run_query("""
            SELECT 
                DATE_TRUNC('month', cluster_date)::date as month,
                COUNT(*) as cluster_count,
                SUM(cluster_size) as total_events
            FROM event_clusters
            GROUP BY DATE_TRUNC('month', cluster_date)
            ORDER BY month
        """)
        print("\nClusters by Month:")
        display(clusters_by_month)
except Exception as e:
    print(f"Event clusters table not available or empty: {e}")

In [ ]:
print("\n" + "=" * 60)
print("CANONICAL EVENTS ANALYSIS")
print("=" * 60)

try:
    canonical_count = run_scalar("SELECT COUNT(*) FROM canonical_events")
    print(f"\nTotal Canonical Events: {canonical_count:,}")
    
    if canonical_count > 0:
        # Master vs child events
        master_count = run_scalar("SELECT COUNT(*) FROM canonical_events WHERE master_event_id IS NULL")
        child_count = run_scalar("SELECT COUNT(*) FROM canonical_events WHERE master_event_id IS NOT NULL")
        print(f"  Master events: {master_count:,}")
        print(f"  Child events: {child_count:,}")
        
        # By country
        canonical_by_country = run_query("""
            SELECT 
                initiating_country,
                COUNT(*) as event_count,
                SUM(total_articles) as total_articles,
                AVG(total_mention_days) as avg_mention_days,
                MIN(first_mention_date) as earliest_mention,
                MAX(last_mention_date) as latest_mention
            FROM canonical_events
            GROUP BY initiating_country
            ORDER BY event_count DESC
        """)
        canonical_by_country['in_config'] = canonical_by_country['initiating_country'].isin(INFLUENCERS)
        print("\nCanonical Events by Country:")
        display(canonical_by_country)
        
        # Story phase distribution
        story_phase = run_query("""
            SELECT 
                story_phase,
                COUNT(*) as event_count
            FROM canonical_events
            GROUP BY story_phase
            ORDER BY event_count DESC
        """)
        print("\nStory Phase Distribution:")
        display(story_phase)
        
        # Materiality scores
        materiality = run_query("""
            SELECT 
                initiating_country,
                COUNT(*) as events_with_score,
                AVG(material_score) as avg_score,
                MIN(material_score) as min_score,
                MAX(material_score) as max_score
            FROM canonical_events
            WHERE material_score IS NOT NULL
            GROUP BY initiating_country
            ORDER BY avg_score DESC
        """)
        print("\nMateriality Scores by Country:")
        display(materiality)
except Exception as e:
    print(f"Canonical events table not available or empty: {e}")

In [ ]:
print("\n" + "=" * 60)
print("DAILY EVENT MENTIONS ANALYSIS")
print("=" * 60)

try:
    mention_count = run_scalar("SELECT COUNT(*) FROM daily_event_mentions")
    print(f"\nTotal Daily Mentions: {mention_count:,}")
    
    if mention_count > 0:
        # By country
        mentions_by_country = run_query("""
            SELECT 
                initiating_country,
                COUNT(*) as mention_count,
                SUM(article_count) as total_articles,
                COUNT(DISTINCT mention_date) as unique_dates,
                MIN(mention_date) as earliest_date,
                MAX(mention_date) as latest_date
            FROM daily_event_mentions
            GROUP BY initiating_country
            ORDER BY mention_count DESC
        """)
        mentions_by_country['in_config'] = mentions_by_country['initiating_country'].isin(INFLUENCERS)
        print("\nDaily Mentions by Country:")
        display(mentions_by_country)
        
        # By month
        mentions_by_month = run_query("""
            SELECT 
                DATE_TRUNC('month', mention_date)::date as month,
                COUNT(*) as mention_count,
                SUM(article_count) as total_articles
            FROM daily_event_mentions
            GROUP BY DATE_TRUNC('month', mention_date)
            ORDER BY month
        """)
        print("\nDaily Mentions by Month:")
        display(mentions_by_month)
        
        # News intensity distribution
        intensity = run_query("""
            SELECT 
                news_intensity,
                COUNT(*) as mention_count
            FROM daily_event_mentions
            GROUP BY news_intensity
            ORDER BY mention_count DESC
        """)
        print("\nNews Intensity Distribution:")
        display(intensity)
except Exception as e:
    print(f"Daily event mentions table not available or empty: {e}")

In [ ]:
print("\n" + "=" * 60)
print("EVENT SUMMARIES ANALYSIS")
print("=" * 60)

try:
    summary_count = run_scalar("SELECT COUNT(*) FROM event_summaries")
    print(f"\nTotal Event Summaries: {summary_count:,}")
    
    if summary_count > 0:
        # By period type
        summaries_by_type = run_query("""
            SELECT 
                period_type,
                COUNT(*) as summary_count,
                MIN(period_start) as earliest_period,
                MAX(period_end) as latest_period
            FROM event_summaries
            GROUP BY period_type
            ORDER BY summary_count DESC
        """)
        print("\nSummaries by Period Type:")
        display(summaries_by_type)
        
        # By country
        summaries_by_country = run_query("""
            SELECT 
                initiating_country,
                COUNT(*) as summary_count,
                SUM(total_documents_across_sources) as total_docs,
                MIN(period_start) as earliest_period,
                MAX(period_end) as latest_period
            FROM event_summaries
            GROUP BY initiating_country
            ORDER BY summary_count DESC
        """)
        summaries_by_country['in_config'] = summaries_by_country['initiating_country'].isin(INFLUENCERS)
        print("\nSummaries by Country:")
        display(summaries_by_country)
        
        # Status distribution
        status_dist = run_query("""
            SELECT 
                status,
                COUNT(*) as summary_count
            FROM event_summaries
            GROUP BY status
        """)
        print("\nStatus Distribution:")
        display(status_dist)
except Exception as e:
    print(f"Event summaries table not available or empty: {e}")

---
## 5. Embeddings Analysis

In [ ]:
print("=" * 60)
print("EMBEDDINGS ANALYSIS")
print("=" * 60)

try:
    # Collections
    collections = run_query("""
        SELECT 
            c.name as collection_name,
            c.uuid,
            COUNT(e.uuid) as embedding_count
        FROM langchain_pg_collection c
        LEFT JOIN langchain_pg_embedding e ON c.uuid = e.collection_id
        GROUP BY c.name, c.uuid
        ORDER BY embedding_count DESC
    """)
    
    total_embeddings = collections['embedding_count'].sum()
    print(f"\nTotal Embedding Collections: {len(collections)}")
    print(f"Total Embeddings: {total_embeddings:,}")
    print("\nCollections:")
    display(collections)
except Exception as e:
    print(f"Embeddings tables not available: {e}")

---
## 6. Country-Focused Analysis (Influencers)

In [ ]:
print("=" * 60)
print("INFLUENCER COUNTRY DEEP DIVE")
print("=" * 60)

for country in INFLUENCERS:
    print(f"\n{'=' * 40}")
    print(f"  {country.upper()}")
    print(f"{'=' * 40}")
    
    # Documents
    doc_stats = run_query("""
        SELECT 
            COUNT(*) as total_docs,
            MIN(date) as earliest_date,
            MAX(date) as latest_date,
            COUNT(DISTINCT date) as unique_dates
        FROM documents
        WHERE initiating_country = :country
    """, {'country': country})
    print(f"\n  Documents: {doc_stats['total_docs'].iloc[0]:,}")
    print(f"  Date Range: {doc_stats['earliest_date'].iloc[0]} to {doc_stats['latest_date'].iloc[0]}")
    
    # Top recipients
    top_recipients = run_query("""
        SELECT 
            recipient_country,
            COUNT(*) as doc_count
        FROM documents
        WHERE initiating_country = :country
        AND recipient_country IS NOT NULL
        GROUP BY recipient_country
        ORDER BY doc_count DESC
        LIMIT 10
    """, {'country': country})
    print(f"\n  Top 10 Recipient Countries:")
    for _, row in top_recipients.iterrows():
        in_config = '(*)' if row['recipient_country'] in RECIPIENTS else ''
        print(f"    {row['recipient_country']}: {row['doc_count']:,} {in_config}")
    
    # Categories
    categories = run_query("""
        SELECT 
            category,
            COUNT(*) as doc_count
        FROM documents
        WHERE initiating_country = :country
        AND category IS NOT NULL
        GROUP BY category
        ORDER BY doc_count DESC
    """, {'country': country})
    print(f"\n  Categories:")
    for _, row in categories.iterrows():
        print(f"    {row['category']}: {row['doc_count']:,}")
    
    # Canonical events (if available)
    try:
        event_stats = run_query("""
            SELECT 
                COUNT(*) as total_events,
                SUM(total_articles) as total_articles,
                AVG(material_score) as avg_materiality
            FROM canonical_events
            WHERE initiating_country = :country
        """, {'country': country})
        print(f"\n  Canonical Events: {event_stats['total_events'].iloc[0]:,}")
        print(f"  Total Articles: {event_stats['total_articles'].iloc[0]:,}")
        if event_stats['avg_materiality'].iloc[0]:
            print(f"  Avg Materiality Score: {event_stats['avg_materiality'].iloc[0]:.2f}")
    except:
        pass

---
## 7. Country-Focused Analysis (Recipients)

In [ ]:
print("=" * 60)
print("RECIPIENT COUNTRY ANALYSIS")
print("=" * 60)

# Summary for all config recipients
recipient_summary = []

for country in RECIPIENTS:
    stats = run_query("""
        SELECT 
            COUNT(*) as total_docs,
            COUNT(DISTINCT initiating_country) as unique_influencers,
            MIN(date) as earliest_date,
            MAX(date) as latest_date
        FROM documents
        WHERE recipient_country = :country
    """, {'country': country})
    
    recipient_summary.append({
        'recipient_country': country,
        'total_docs': stats['total_docs'].iloc[0],
        'unique_influencers': stats['unique_influencers'].iloc[0],
        'earliest_date': stats['earliest_date'].iloc[0],
        'latest_date': stats['latest_date'].iloc[0]
    })

recipient_df = pd.DataFrame(recipient_summary)
recipient_df = recipient_df.sort_values('total_docs', ascending=False)
print("\nRecipient Countries Summary:")
recipient_df

In [ ]:
# Influencer activity in top recipient countries
print("\n" + "=" * 60)
print("INFLUENCER ACTIVITY BY RECIPIENT")
print("=" * 60)

top_recipients = recipient_df.head(10)['recipient_country'].tolist()

for recipient in top_recipients:
    print(f"\n  {recipient}:")
    influencer_activity = run_query("""
        SELECT 
            initiating_country,
            COUNT(*) as doc_count
        FROM documents
        WHERE recipient_country = :recipient
        AND initiating_country IN :influencers
        GROUP BY initiating_country
        ORDER BY doc_count DESC
    """, {'recipient': recipient, 'influencers': tuple(INFLUENCERS)})
    
    for _, row in influencer_activity.iterrows():
        print(f"    {row['initiating_country']}: {row['doc_count']:,}")

---
## 8. Bilateral Relationship Summaries

In [ ]:
print("=" * 60)
print("BILATERAL RELATIONSHIP SUMMARIES")
print("=" * 60)

try:
    bilateral_count = run_scalar("SELECT COUNT(*) FROM bilateral_relationship_summaries")
    print(f"\nTotal Bilateral Summaries: {bilateral_count:,}")
    
    if bilateral_count > 0:
        bilateral_summaries = run_query("""
            SELECT 
                initiating_country,
                recipient_country,
                total_documents,
                total_daily_events,
                material_score_avg,
                first_interaction_date,
                last_interaction_date
            FROM bilateral_relationship_summaries
            WHERE is_deleted = false
            ORDER BY total_documents DESC
            LIMIT 20
        """)
        print("\nTop 20 Bilateral Relationships by Document Count:")
        display(bilateral_summaries)
except Exception as e:
    print(f"Bilateral summaries table not available: {e}")

---
## 9. Data Quality Checks

In [ ]:
print("=" * 60)
print("DATA QUALITY CHECKS")
print("=" * 60)

# Null checks for documents
null_checks = run_query("""
    SELECT 
        COUNT(*) as total_docs,
        SUM(CASE WHEN date IS NULL THEN 1 ELSE 0 END) as null_dates,
        SUM(CASE WHEN initiating_country IS NULL THEN 1 ELSE 0 END) as null_init_country,
        SUM(CASE WHEN recipient_country IS NULL THEN 1 ELSE 0 END) as null_rec_country,
        SUM(CASE WHEN category IS NULL THEN 1 ELSE 0 END) as null_category,
        SUM(CASE WHEN salience_bool IS NULL THEN 1 ELSE 0 END) as null_salience
    FROM documents
""")

print("\nNull Value Analysis (Documents):")
total = null_checks['total_docs'].iloc[0]
print(f"  Total documents: {total:,}")
print(f"  Null dates: {null_checks['null_dates'].iloc[0]:,} ({null_checks['null_dates'].iloc[0]/total*100:.1f}%)")
print(f"  Null initiating_country: {null_checks['null_init_country'].iloc[0]:,} ({null_checks['null_init_country'].iloc[0]/total*100:.1f}%)")
print(f"  Null recipient_country: {null_checks['null_rec_country'].iloc[0]:,} ({null_checks['null_rec_country'].iloc[0]/total*100:.1f}%)")
print(f"  Null category: {null_checks['null_category'].iloc[0]:,} ({null_checks['null_category'].iloc[0]/total*100:.1f}%)")
print(f"  Null salience_bool: {null_checks['null_salience'].iloc[0]:,} ({null_checks['null_salience'].iloc[0]/total*100:.1f}%)")

# Orphan checks
print("\nOrphan Record Checks:")
try:
    orphan_categories = run_scalar("""
        SELECT COUNT(*) FROM categories c
        WHERE NOT EXISTS (SELECT 1 FROM documents d WHERE d.doc_id = c.doc_id)
    """)
    print(f"  Orphan categories: {orphan_categories:,}")
except:
    print("  Orphan categories: N/A")

try:
    orphan_mentions = run_scalar("""
        SELECT COUNT(*) FROM daily_event_mentions m
        WHERE NOT EXISTS (SELECT 1 FROM canonical_events e WHERE e.id = m.canonical_event_id)
    """)
    print(f"  Orphan daily mentions: {orphan_mentions:,}")
except:
    print("  Orphan daily mentions: N/A")

---
## 10. Summary Statistics

In [ ]:
print("=" * 60)
print("EXECUTIVE SUMMARY")
print("=" * 60)

summary = {
    'metric': [],
    'value': []
}

# Documents
summary['metric'].append('Total Documents')
summary['value'].append(f"{run_scalar('SELECT COUNT(*) FROM documents'):,}")

# Date range
date_info = run_query("SELECT MIN(date), MAX(date) FROM documents WHERE date IS NOT NULL")
summary['metric'].append('Date Range')
summary['value'].append(f"{date_info.iloc[0, 0]} to {date_info.iloc[0, 1]}")

# Influencer coverage
influencer_docs = run_scalar(f"""
    SELECT COUNT(*) FROM documents 
    WHERE initiating_country IN {tuple(INFLUENCERS)}
""")
summary['metric'].append('Documents from Config Influencers')
summary['value'].append(f"{influencer_docs:,}")

# Recipient coverage
recipient_docs = run_scalar(f"""
    SELECT COUNT(*) FROM documents 
    WHERE recipient_country IN {tuple(RECIPIENTS)}
""")
summary['metric'].append('Documents to Config Recipients')
summary['value'].append(f"{recipient_docs:,}")

# Events
try:
    summary['metric'].append('Canonical Events')
    summary['value'].append(f"{run_scalar('SELECT COUNT(*) FROM canonical_events'):,}")
except:
    pass

try:
    summary['metric'].append('Event Clusters')
    summary['value'].append(f"{run_scalar('SELECT COUNT(*) FROM event_clusters'):,}")
except:
    pass

try:
    summary['metric'].append('Daily Event Mentions')
    summary['value'].append(f"{run_scalar('SELECT COUNT(*) FROM daily_event_mentions'):,}")
except:
    pass

try:
    summary['metric'].append('Event Summaries')
    summary['value'].append(f"{run_scalar('SELECT COUNT(*) FROM event_summaries'):,}")
except:
    pass

# Embeddings
try:
    summary['metric'].append('Total Embeddings')
    summary['value'].append(f"{run_scalar('SELECT COUNT(*) FROM langchain_pg_embedding'):,}")
except:
    pass

summary_df = pd.DataFrame(summary)
print("\n")
summary_df

In [ ]:
print("\n" + "=" * 60)
print("AUDIT COMPLETE")
print("=" * 60)
print(f"\nGenerated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")